In [3]:
!pip install xgboost -q

In [1]:
import polars as pl                                                                                                                                         
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Загрузка
train_main = pl.read_parquet('../data/raw/train_main_features.parquet')
train_extra = pl.read_parquet('../data/raw/train_extra_features.parquet')
target = pl.read_parquet('../data/raw/train_target.parquet')

target_columns = [col for col in target.columns if col.startswith('target')]

# Статистика по extra
extra_cols = [c for c in train_extra.columns if c != 'customer_id']
print(f'Main features: {len(train_main.columns) - 1}')
print(f'Extra features: {len(extra_cols)}')
print(f'Всего: {len(train_main.columns) - 1 + len(extra_cols)}')

# Пропуски в extra
null_counts = train_extra.select(extra_cols).null_count()
null_pcts = {col: null_counts[col][0] / len(train_extra) * 100 for col in extra_cols}

# Распределение пропусков
bins = [0, 1, 10, 30, 50, 70, 90, 99, 100]
labels = ['0-1%', '1-10%', '10-30%', '30-50%', '50-70%', '70-90%', '90-99%', '99-100%']
hist = [0] * len(labels)
for pct in null_pcts.values():
    for i in range(len(bins) - 1):
        if bins[i] <= pct < bins[i+1]:
            hist[i] += 1
            break

print(f'\nПропуски в extra features:')
for label, count in zip(labels, hist):
    bar = '█' * (count // 10) + '▌' * (1 if count % 10 >= 5 else 0)
    print(f'  {label:>8s}: {count:4d} фичей  {bar}')

# Сколько фичей без пропусков
no_nulls = sum(1 for p in null_pcts.values() if p == 0)
heavy_nulls = sum(1 for p in null_pcts.values() if p > 90)
print(f'\nБез пропусков: {no_nulls}')
print(f'Пропусков >90%: {heavy_nulls}')

Main features: 199
Extra features: 2241
Всего: 2440

Пропуски в extra features:
      0-1%:   12 фичей  █
     1-10%:    9 фичей  ▌
    10-30%:  774 фичей  █████████████████████████████████████████████████████████████████████████████
    30-50%:  322 фичей  ████████████████████████████████
    50-70%:  167 фичей  ████████████████▌
    70-90%:  386 фичей  ██████████████████████████████████████▌
    90-99%:  275 фичей  ███████████████████████████▌
   99-100%:  296 фичей  █████████████████████████████▌

Без пропусков: 0
Пропусков >90%: 571


In [4]:
import xgboost as xgb                                                                                                                                       
                                                                                                                                                              
# Объединяем main + extra                                                                                                                                 
train_full = train_main.join(train_extra, on='customer_id', how='left')                                                                                     
cat_features = [col for col in train_main.columns if col.startswith('cat_feature')]                                                                         
train_full = train_full.with_columns(pl.col(cat_features).cast(pl.Int32))                                                                                   
feature_columns = [col for col in train_full.columns if col != 'customer_id']

# Подвыборка 100k
np.random.seed(42)
idx = np.random.choice(len(train_full), 100000, replace=False)
X_quick = train_full[idx].select(feature_columns).to_pandas()
y_quick = target[idx]['target_8_1'].to_pandas()

# Быстрое обучение
dtrain = xgb.DMatrix(X_quick, label=y_quick)
model = xgb.train({
    'objective': 'binary:logistic',
    'max_depth': 6, 'learning_rate': 0.05,
    'subsample': 0.8, 'colsample_bytree': 0.8,
    'tree_method': 'hist', 'device': 'cuda',
    'seed': 42, 'verbosity': 0
}, dtrain, num_boost_round=500)

# Feature importance (gain — суммарный прирост качества от фичи)
importance = model.get_score(importance_type='gain')

# Статистика
used = len(importance)
unused = len(feature_columns) - used
print(f'Используется: {used} из {len(feature_columns)} ({used/len(feature_columns)*100:.0f}%)')
print(f'Не используется: {unused} ({unused/len(feature_columns)*100:.0f}%)')

# Разбивка по группам пропусков
used_set = set(importance.keys())
for label, lo, hi in [('0-30%', 0, 30), ('30-70%', 30, 70), ('70-90%', 70, 90), ('90-100%', 90, 100)]:
    group = [c for c in extra_cols if lo <= null_pcts[c] < hi]
    group_used = [c for c in group if c in used_set]
    print(f'\n{label} пропусков ({len(group)} фичей):')
    print(f'  Используется: {len(group_used)} ({len(group_used)/max(len(group),1)*100:.0f}%)')

# Топ-20 фичей
top20 = sorted(importance.items(), key=lambda x: -x[1])[:20]
print(f'\nТоп-20 по importance:')
for i, (feat, gain) in enumerate(top20):
    null_pct = null_pcts.get(feat, 0)
    src = 'main' if feat.startswith(('cat_', 'num_')) and feat in train_main.columns else 'extra'
    print(f'  {i+1:2d}. {feat:25s} | gain: {gain:10.1f} | nulls: {null_pct:5.1f}% | {src}')

Используется: 1318 из 2440 (54%)
Не используется: 1122 (46%)

0-30% пропусков (795 фичей):
  Используется: 393 (49%)

30-70% пропусков (489 фичей):
  Используется: 281 (57%)

70-90% пропусков (386 фичей):
  Используется: 314 (81%)

90-100% пропусков (571 фичей):
  Используется: 212 (37%)

Топ-20 по importance:
   1. num_feature_22            | gain:     1177.4 | nulls:   0.0% | main
   2. num_feature_27            | gain:      223.3 | nulls:   0.0% | main
   3. num_feature_62            | gain:      191.8 | nulls:   0.0% | main
   4. num_feature_7             | gain:      167.9 | nulls:   0.0% | main
   5. num_feature_1847          | gain:      162.6 | nulls:  29.7% | extra
   6. num_feature_176           | gain:       76.9 | nulls:  24.1% | extra
   7. num_feature_1429          | gain:       67.4 | nulls:  59.6% | extra
   8. num_feature_1549          | gain:       60.7 | nulls:  44.9% | extra
   9. num_feature_436           | gain:       56.5 | nulls:  44.5% | extra
  10. num_feature

In [6]:
import xgboost as xgb                                                                                                                                       
import numpy as np                                                                                                                                          
import pandas as pd                                                                                                                                         
                                                                                                                                                            
# === Загрузка данных ===                                                                                                                                   
DATA = '/home/masked/projects/data_fusion/data/raw'                                                                                                       

train_main = pd.read_parquet(f'{DATA}/train_main_features.parquet')
train_extra = pd.read_parquet(f'{DATA}/train_extra_features.parquet')
train_target = pd.read_parquet(f'{DATA}/train_target.parquet')

X_full = train_main.merge(train_extra, on='customer_id', how='left')
customer_ids = X_full['customer_id']
X_full = X_full.drop(columns=['customer_id'])

# Категориальные → Int32
cat_cols = [c for c in X_full.columns if c.startswith('cat_feature')]
for c in cat_cols:
    X_full[c] = X_full[c].astype('Int32')

y_full = train_target.drop(columns=['customer_id'])

# Подвыборка 100k для быстрого теста
np.random.seed(42)
idx = np.random.choice(len(X_full), 100_000, replace=False)
X_sample = X_full.iloc[idx].reset_index(drop=True)
y_sample = y_full.iloc[idx].reset_index(drop=True)

print(f"X_sample: {X_sample.shape}, y_sample: {y_sample.shape}")

# === Importance на 4 таргетах ===
targets_to_check = {
    'target_8_1': 'сильный',
    'target_9_6': 'слабый (частый 22.3%)',
    'target_3_1': 'слабый',
    'target_1_1': 'средний',
}

params = {
    'objective': 'binary:logistic', 'eval_metric': 'auc',
    'learning_rate': 0.05, 'max_depth': 6, 'min_child_weight': 5,
    'subsample': 0.8, 'colsample_bytree': 0.8,
    'tree_method': 'hist', 'device': 'cuda', 'seed': 42, 'verbosity': 0,
}

all_importances = {}

for target_name, desc in targets_to_check.items():
    print(f"\n{'='*60}")
    print(f"Обучаем XGBoost на {target_name} ({desc})")

    y_target = y_sample[target_name].values
    n_pos = int(y_target.sum())
    print(f"  Positives: {n_pos} ({n_pos/len(y_target)*100:.1f}%)")

    dtrain = xgb.DMatrix(X_sample, label=y_target, enable_categorical=True)
    model = xgb.train(params, dtrain, num_boost_round=500, verbose_eval=False)

    imp = model.get_score(importance_type='gain')
    all_importances[target_name] = set(imp.keys())
    print(f"  Используемых фичей: {len(imp)}")

# === Анализ пересечений ===
print(f"\n{'='*60}")
print("АНАЛИЗ ПЕРЕСЕЧЕНИЙ")
print(f"{'='*60}")

for name, feats in all_importances.items():
    print(f"  {name}: {len(feats)} фичей")

core_features = set.intersection(*all_importances.values())
print(f"\nОбщее ядро (все 4 таргета): {len(core_features)} фичей")

any_used = set.union(*all_importances.values())
print(f"Используются хотя бы одной: {len(any_used)} фичей")

all_features = set(X_sample.columns)
never_used = all_features - any_used
print(f"Никем не используются: {len(never_used)} фичей — кандидаты на удаление")

X_sample: (100000, 2440), y_sample: (100000, 41)

Обучаем XGBoost на target_8_1 (сильный)
  Positives: 10401 (10.4%)
  Используемых фичей: 1171

Обучаем XGBoost на target_9_6 (слабый (частый 22.3%))
  Positives: 22158 (22.2%)
  Используемых фичей: 1424

Обучаем XGBoost на target_3_1 (слабый)
  Positives: 9887 (9.9%)
  Используемых фичей: 1381

Обучаем XGBoost на target_1_1 (средний)
  Positives: 1007 (1.0%)
  Используемых фичей: 1031

АНАЛИЗ ПЕРЕСЕЧЕНИЙ
  target_8_1: 1171 фичей
  target_9_6: 1424 фичей
  target_3_1: 1381 фичей
  target_1_1: 1031 фичей

Общее ядро (все 4 таргета): 831 фичей
Используются хотя бы одной: 1688 фичей
Никем не используются: 752 фичей — кандидаты на удаление


In [7]:
# Ячейка 4: A/B тест — все фичи vs без 752 мусорных                                                                                                         
from sklearn.model_selection import train_test_split                                                                                                        
from sklearn.metrics import roc_auc_score                                                                                                                   
                                                                                                                                                            
# Сплит 80/20 для честной оценки                                                                                                                            
X_tr, X_vl, y_tr, y_vl = train_test_split(                                                                                                                  
    X_sample, y_sample, test_size=0.2, random_state=42                                                                                                    
)

# Список фичей без 752 мусорных
clean_features = sorted(list(any_used))  # 1688 фичей
print(f"Все фичи: {X_tr.shape[1]}")
print(f"Без мусора: {len(clean_features)}")
print(f"Удалено: {X_tr.shape[1] - len(clean_features)}")

params = {
    'objective': 'binary:logistic', 'eval_metric': 'auc',
    'learning_rate': 0.05, 'max_depth': 6, 'min_child_weight': 5,
    'subsample': 0.8, 'colsample_bytree': 0.8,
    'tree_method': 'hist', 'device': 'cuda', 'seed': 42, 'verbosity': 0,
}

targets = ['target_8_1', 'target_9_6', 'target_3_1', 'target_1_1']

print(f"\n{'Таргет':<15} {'Все 2440':>10} {'1688 фичей':>12} {'Δ AUC':>10}")
print("-" * 50)

for target in targets:
    y_t = y_tr[target].values
    y_v = y_vl[target].values

    # Все фичи
    dtrain_all = xgb.DMatrix(X_tr, label=y_t, enable_categorical=True)
    dval_all = xgb.DMatrix(X_vl, label=y_v, enable_categorical=True)
    model_all = xgb.train(params, dtrain_all, num_boost_round=500, verbose_eval=False)
    auc_all = roc_auc_score(y_v, model_all.predict(dval_all))

    # Без мусора
    dtrain_clean = xgb.DMatrix(X_tr[clean_features], label=y_t, enable_categorical=True)
    dval_clean = xgb.DMatrix(X_vl[clean_features], label=y_v, enable_categorical=True)
    model_clean = xgb.train(params, dtrain_clean, num_boost_round=500, verbose_eval=False)
    auc_clean = roc_auc_score(y_v, model_clean.predict(dval_clean))

    delta = auc_clean - auc_all
    sign = "+" if delta >= 0 else ""
    print(f"{target:<15} {auc_all:>10.4f} {auc_clean:>12.4f} {sign}{delta:>9.4f}")

Все фичи: 2440
Без мусора: 1688
Удалено: 752

Таргет            Все 2440   1688 фичей      Δ AUC
--------------------------------------------------
target_8_1          0.9825       0.9827 +   0.0003
target_9_6          0.6860       0.6841   -0.0019
target_3_1          0.6778       0.6777   -0.0000
target_1_1          0.8900       0.8835   -0.0064


In [ ]:
# Ячейка 5: Per-target feature selection через Cumulative Gain (Исправленная)
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import xgboost as xgb
import warnings

# Отключаем ворнинги от pandas при срезах
warnings.filterwarnings("ignore")

# 8 репрезентативных таргетов
targets = [
    'target_2_2', 'target_8_1',   # сильные
    'target_1_1', 'target_7_2',   # средние
    'target_9_6', 'target_3_1',   # слабые
    'target_2_8', 'target_6_5',   # редкие
]

GAIN_THRESHOLD = 0.95  # оставляем фичи дающие 95% суммарного gain

print(f"{'Таргет':<15} {'n_pos':>7} {'Все 2440':>10} {'CumGain95':>11} {'N фичей':>9} {'Δ AUC':>8}")
print("-" * 65)

for target in targets:
    # 1. Стратифицированный сплит ДЛЯ КОНКРЕТНОГО ТАРГЕТА!
    # Это гарантирует, что редкие единички попадут и в трейн, и в валидацию
    X_tr, X_vl, y_tr, y_vl = train_test_split(
        X_sample, y_sample[target], test_size=0.2, random_state=42, stratify=y_sample[target]
    )
    
    y_t = y_tr.values
    y_v = y_vl.values
    n_pos = int(y_t.sum())

    # 2. Динамический min_child_weight под редкость таргета
    # Если мало позитивов, разрешаем дереву делать сплиты на единичных примерах
    current_min_child = 5 if n_pos > 500 else 1
    
    params = {
        'objective': 'binary:logistic', 'eval_metric': 'auc',
        'learning_rate': 0.05, 'max_depth': 6, 'min_child_weight': current_min_child,
        'subsample': 0.8, 'colsample_bytree': 0.8,
        'tree_method': 'hist', 'device': 'cuda', 'seed': 42, 'verbosity': 0,
    }

    # 3. Базовая модель на всех фичах
    dtrain = xgb.DMatrix(X_tr, label=y_t, enable_categorical=True)
    dval = xgb.DMatrix(X_vl, label=y_v, enable_categorical=True)
    model = xgb.train(params, dtrain, num_boost_round=500, verbose_eval=False)
    
    # Защита от метрики на пустом валидационном сете (для ультра-редких)
    try:
        auc_all = roc_auc_score(y_v, model.predict(dval))
    except ValueError:
        auc_all = 0.5 

    # 4. Извлекаем importance, сортируем по gain
    imp = model.get_score(importance_type='gain')
    imp_sorted = sorted(imp.items(), key=lambda x: x[1], reverse=True)

    # 5. ЗАЩИТА ОТ КРАША: Если модель не смогла сделать сплиты (пустой importance)
    if not imp_sorted:
        print(f"{target:<15} {n_pos:>7} {auc_all:>10.4f} {'FAIL':>11} {'ALL':>9}  Дерево не строится")
        continue

    # 6. Cumulative gain — находим порог 95%
    total_gain = sum(v for _, v in imp_sorted)
    cum_gain = 0
    selected_features = []
    
    for feat, gain in imp_sorted:
        cum_gain += gain
        selected_features.append(feat)
        if cum_gain / total_gain >= GAIN_THRESHOLD:
            break
            
    # Минимальная защита: оставляем хотя бы 10 фичей, даже если порог пройден раньше
    if len(selected_features) < 10:
        selected_features = [f for f, _ in imp_sorted[:10]]

    # 7. Обучаем на отобранных фичах
    dtrain_sel = xgb.DMatrix(X_tr[selected_features], label=y_t, enable_categorical=True)
    dval_sel = xgb.DMatrix(X_vl[selected_features], label=y_v, enable_categorical=True)
    model_sel = xgb.train(params, dtrain_sel, num_boost_round=500, verbose_eval=False)
    
    try:
        auc_sel = roc_auc_score(y_v, model_sel.predict(dval_sel))
    except ValueError:
        auc_sel = 0.5
        
    delta = auc_sel - auc_all
    sign = "+" if delta >= 0 else ""
    print(f"{target:<15} {n_pos:>7} {auc_all:>10.4f} {auc_sel:>11.4f} {len(selected_features):>9} {sign}{delta:>7.4f}")

Таргет            n_pos   Все 2440   CumGain95   N фичей    Δ AUC
-----------------------------------------------------------------
target_2_2         2006     0.9311      0.9335       979 + 0.0024
target_8_1         8321     0.9804      0.9807       970 + 0.0002
target_1_1          806     0.8729      0.8714       812 -0.0015
target_7_2         2184     0.8172      0.8199      1003 + 0.0027
target_9_6        17726     0.6735      0.6772      1282 + 0.0036
target_3_1         7910     0.6786      0.6788      1239 + 0.0001
target_2_8            9     0.9997      0.9997        84 -0.0000
target_6_5           40     0.9153      0.8927       334 -0.0227


: 